# Your Next Query: Empty Parking Space Detection

Now let's try another query - empty parking space detection. You'll use Rekall to detect empty parking spaces in a fixed-angle camera feed of a parking lot -- using nothing more than the outputs of an off-the-shelf object detector!

We'll start by importing Rekall and loading up the data:

In [ ]:
from rekall import Interval, IntervalSet, IntervalSetMapping, Bounds3D
from rekall.predicates import *
from vgrid import VGridSpec, VideoMetadata, VideoBlockFormat, FlatFormat, SpatialType_Bbox
from vgrid_jupyter import VGridWidget
import urllib3, requests, os
import pickle
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt

# Hack to disable warnings about olimar's certificate
urllib3.disable_warnings()

# Parking lot data
VIDEO_COLLECTION_BASEURL = "https://olimar.stanford.edu/hdd/parking_lot/user_study"
VIDEO_METADATA_FILENAME = 'metadata.json'

# Load video file metadata
video_metadata = [ VideoMetadata(v['filename'], id=v['id'], fps=v['fps'],
                                 num_frames=v['num_frames'], width=v['width'],
                                 height=v['height'])
                  for v in requests.get(os.path.join(
                      VIDEO_COLLECTION_BASEURL, VIDEO_METADATA_FILENAME),
                                        verify=False).json() ]

VIDEO_FOLDER = 'videos'
BBOX_FOLDER = 'bboxes'
GT_FOLDER = 'empty_spaces'

dev_set = requests.get(
    os.path.join(VIDEO_COLLECTION_BASEURL, 'dev.txt'), verify=False
).content.decode('utf-8').strip().split('\n')

video_metadata_dev = [
    vm
    for vm in video_metadata if vm.path in dev_set
]

video_metadata = video_metadata_dev

interval = 30
bboxes = [
    pickle.loads(requests.get(
        os.path.join(
            os.path.join(VIDEO_COLLECTION_BASEURL, BBOX_FOLDER),
            os.path.join(vm.path[:-4], 'bboxes.pkl')
        ),
        verify=False
    ).content)
    for vm in (video_metadata)
]
bboxes_ism = IntervalSetMapping({
    metadata.id: IntervalSet([
        Interval(
            Bounds3D(
                t1 = 30 * i / metadata.fps,
                t2 = 30 * (i + interval) / metadata.fps,
                x1 = bbox[0] / metadata.width,
                x2 = bbox[2] / metadata.width,
                y1 = bbox[1] / metadata.height,
                y2 = bbox[3] / metadata.height
            ),
            payload = { 'class': bbox[4], 'score': bbox[5] }
        )
        for i, frame in enumerate(bbox_frame_list) if (i % interval == 0)
        for bbox in frame
    ])
    for bbox_frame_list, metadata in tqdm(
        zip(bboxes, (video_metadata)),
        total = len(bboxes))
})

def visualize_helper(box_list):
    vgrid_spec = VGridSpec(
        video_meta = video_metadata,
        vis_format = VideoBlockFormat(imaps = [
            (str(i), box)
            for i, box in enumerate(box_list)
        ]),
        video_endpoint = os.path.join(VIDEO_COLLECTION_BASEURL, VIDEO_FOLDER)
    )
    return VGridWidget(vgrid_spec = vgrid_spec.to_json_compressed())

Let's visualize the data (drawing the class of each object underneath its bounding box using `SpatialType_Bbox`):

In [ ]:
visualize_helper([bboxes_ism.map(
    lambda interval: Interval(
        interval['bounds'],
        { 'spatial_type': SpatialType_Bbox(text=interval['payload']['class']) }
    )
)])

If you click on the second video, you can see that there are sometimes empty parking spaces in this parking lot. Our goal is to detect these by creating an `IntervalSetMapping` object that contains all the empty parking spaces in these videos.

# Task: detect all empty parking spaces

Your goal is to write a Rekall program to detect all empty parking spaces (visualized in the second timeline above).

We're starting with a `IntervalSetMapping` object, `bboxes_ism`, that contains detections from Mask R-CNN. The Intervals contain 3D bounds, and the payloads contain the class and the class score:

In [ ]:
bboxes_ism[0].get_intervals()[0]

The bounding boxes are sampled every thirty seconds (hence why the Interval above has time bounds of 0 to 30), and so are the ground truth annotations.

The goal is to write a query that detects all the **empty parking spaces** in our videos.

This task is inspired by [this Medium blog post](https://medium.com/@ageitgey/snagging-parking-spaces-with-mask-r-cnn-and-python-955f2231c400):
* They use an off-the-shelf object detector to detect cars (like what we have in `bboxes`)
* They take a timestamp where all the parking spots are full, and use car detections to get parking spots
* Then empty parking spots are just parking spots where there are no cars

We'll be following these steps in this tutorial!

# Step 1: Detect Parking Spaces

Before we detect empty parking spaces, we first need to detect parking spaces! Luckily, every parking space is filled at the beginning of the first video. We can use the **car detections** at the beginning of this video to construct an `IntervalSetMapping` object corresponding to parking spaces in both videos.

Let's begin by looking at the video ID's of our videos:

In [ ]:
bboxes_ism.keys()

Videos are sorted by ID, so we know that the video on the left is video 0.

Let's start by getting all the car detections at the beginning of the video. We can reference the `IntervalSet` corresponding to video `0` as follows:

```Python
bboxes_ism[0]
```

And filter down to the car detections at the beginning of video `0` like this:

```Python
parking_spot_candidates = bboxes_ism[0].filter(
    lambda intrvl: intrvl['t1'] == 0.0 and intrvl['payload']['class'] == 'car'
)
```

Try it yourself below!

In [ ]:
# Copy the code segment above


`parking_spot_candidates` contains all the car detections at time `0` of video `0`. Next, we want to create `parking_spots`, an `IntervalSetMapping` object that represents all the parking spots (empty or not) in both of our videos.

We'll need to create a new `IntervalSetMapping` object called `parking_spots` that contains:
* Two keys (`0` and `2`), each of which points to an `IntervalSet` that contains parking spots
* Each `IntervalSet` should have `Intervals` for each parking spot in the parking lot
* Since our object detections are sampled once every 30 seconds, we can sample the parking spot objects once every thirty seconds as well

Go ahead and give this a try now! Solution is below.

In [ ]:
# Construct parking spots!


Exercise solution:

```Python
# Construct parking spots!
video_lengths = {
    key: bboxes_ism[key].get_intervals()[-1]['t2']
    for key in bboxes_ism.keys()
}

parking_spots = IntervalSetMapping({
    key: IntervalSet([
        Interval(Bounds3D(
            t1 = t,
            t2 = t + 30, # Make the interval last 30 seconds
            x1 = parking_spot['x1'],
            x2 = parking_spot['x2'],
            y1 = parking_spot['y1'],
            y2 = parking_spot['y2']
        ))
        for parking_spot in parking_spot_candidates.get_intervals() # For each parking spot
        for t in range(0, int(video_lengths[key]), 30)
    ])
    for key in bboxes_ism.keys()
})
```

Let's take a moment to visualize `parking_spots` and make sure it looks right -- you should have bounding boxes over each parking spot for the duration of the video.

In [ ]:
visualize_helper([parking_spots])

# Step 2: Detect Empty Parking Spaces using `minus`

Now we have `parking_spots`, which represents all the parking spots in our video feeds, and we want to detect the empty parking spots.

This is a great use of Rekall's [`minus`](https://rekallpy.readthedocs.io/en/latest/index.html#rekall.IntervalSet.minus) function:
![simple_minus.png](https://olimar.stanford.edu/hdd/rekall_tutorials/simple_minus.png)

We can use the `minus` function to find all the instances where a parking spot does not have sufficient overlap with a car:

```Python
cars = bboxes_ism.filter(
    lambda intrvl: intrvl['payload']['class'] == 'car'
)

empty_parking_spots = parking_spots.minus(
    cars,
    predicate = and_pred(
        Bounds3D.T(overlaps()),
        Bounds3D.X(overlaps()),
        Bounds3D.Y(overlaps()),
        iou_at_least(0.25)
    ),
    window=0.0,
    progress_bar=True
)
```

This code takes any parking spot detection that overlaps with a detected car Interval with [IOU](https://www.pyimagesearch.com/2016/11/07/intersection-over-union-iou-for-object-detection/) at least 0.5, and removes it from the set. The remaining Intervals are empty parking spots.

Try it yourself below (it'll take about ten seconds)!

In [ ]:
# Fill in the code segments above


And let's visualize the results:

In [ ]:
visualize_helper([empty_parking_spots])

# Step 3: Remove False Positives with `coalesce` and `filter_size`

So this is pretty good! But we have some false positives due to errors in the object detector; sometimes the cars are not detected. This results in parking spots that appear for 30 seconds at a time before disappearing. Let's remove some of these false positives by removing any parking spots that appear for less than a few minutes!

This is a great chance to use Rekall's [`coalesce`](https://rekallpy.readthedocs.io/en/latest/index.html#rekall.IntervalSet.coalesce) function:
![simple_coalesce.png](https://olimar.stanford.edu/hdd/rekall_tutorials/simple_coalesce.png)

We'll use the `coalesce` function to merge all parking spots that are adjacent in time and overlapping in space:

```Python
empty_parking_spots_merged = empty_parking_spots.coalesce(
    ('t1', 't2'),
    bounds_merge_op = Bounds3D.span,
    predicate = iou_at_least(0.5)
)
```

Let's break down what's happening here:
* `coalesce` recursively merges all overlapping or touching Intervals along some axis (in this case, time).
* We merge the bounds by taking the span of the existing Intervals and the new Interval.
* We add in a predicate that we should only merge in new intervals if the IOU is at least 0.5.
* At the end, we have a Interval that covers the entire contiguous spacetime volume of an empty parking spot.

Then we can filter the resulting set by length to remove any parking spots that appear for less than four minutes:
```Python
empty_parking_spots_filtered = empty_parking_spots_merged.filter_size(
    min_size = 60 * 4
)
```

Try it yourself below!

In [ ]:
# Fill in the two code snippets above


This turns out to do pretty well! We have completely removed all the false detections from the first video (hence why it doesn't appear anymore), and we've removed many of the false detections from the second video.

# Step 4: Continue Debugging On Your Own

Now it's your turn! Can you modify the above code to get rid of even more false positives? Here's a few hints:
* We've fixed some issues with objects not being detected using the `coalesce` and `filter_size` functions, but what other issues might the object detector have?
* In particular, are there cases where a car could be mis-classified as something else? Take a look at the initial visualization of objects to get an idea.
* Try visualizing object classes like people, cars, and trucks to see what could be going on...
* This task was discussed in the [Rekall tech report](http://www.danfu.org/projects/rekall-tech-report/) (with pseudocode), so take a look at that for our solution!

In [ ]:
# Play around on your own!
